In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model_finbert = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <B3E58761-2785-34C6-A89B-F37110C88A05> /opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <3BD7C270-CE25-3DCF-A6C8-B9D7148F8ADE> /opt/homebrew/Caskroom/miniforge/base/envs/myenv/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# Assuming model_finbert is a PyTorch model
import torch
total_params = sum(p.numel() for p in model_finbert.parameters())
print(f'Total number of parameters in model_finbert: {total_params}')

Total number of parameters in model_finbert: 109484547


In [4]:
import scipy
import torch.nn.functional as F

tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
x = ["DJ index increases!", "DJ index to decrease by 10%"]
input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
logits = model_finbert(**input_sequence).logits
probabilities = F.softmax(logits, dim=-1)
print(probabilities)

tensor([[0.5843, 0.0742, 0.3416],
        [0.0092, 0.9704, 0.0204]], grad_fn=<SoftmaxBackward0>)


In [5]:
from data_preparation_utilities import read_csv_from_zip
Combined_News_DJIA, DJIA_table, RedditNews, Test_dates = read_csv_from_zip()

In [6]:
Combined_News_DJIA = Combined_News_DJIA.drop(columns=['Date', 'Label'], axis=1)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [8]:
import scipy
import torch.nn.functional as F
import torch
tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
Combined_News_DJIA_np = Combined_News_DJIA.to_numpy()
n = Combined_News_DJIA_np.shape[0]
final = torch.tensor([])
for i in range(50):
    j = 0
    x = [char for char in Combined_News_DJIA_np[i].tolist()]
    input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
    logits = model_finbert(**input_sequence).logits
    probabilities = F.softmax(logits, dim=-1).view(1, 75)
    final = torch.cat((final, probabilities), dim=0)
    print(final.shape)

torch.Size([1, 75])
torch.Size([2, 75])
torch.Size([3, 75])
torch.Size([4, 75])
torch.Size([5, 75])
torch.Size([6, 75])
torch.Size([7, 75])
torch.Size([8, 75])
torch.Size([9, 75])
torch.Size([10, 75])
torch.Size([11, 75])
torch.Size([12, 75])
torch.Size([13, 75])
torch.Size([14, 75])
torch.Size([15, 75])
torch.Size([16, 75])
torch.Size([17, 75])
torch.Size([18, 75])
torch.Size([19, 75])
torch.Size([20, 75])
torch.Size([21, 75])
torch.Size([22, 75])
torch.Size([23, 75])
torch.Size([24, 75])
torch.Size([25, 75])
torch.Size([26, 75])
torch.Size([27, 75])
torch.Size([28, 75])
torch.Size([29, 75])
torch.Size([30, 75])
torch.Size([31, 75])
torch.Size([32, 75])
torch.Size([33, 75])
torch.Size([34, 75])
torch.Size([35, 75])
torch.Size([36, 75])
torch.Size([37, 75])
torch.Size([38, 75])
torch.Size([39, 75])
torch.Size([40, 75])
torch.Size([41, 75])
torch.Size([42, 75])
torch.Size([43, 75])
torch.Size([44, 75])
torch.Size([45, 75])
torch.Size([46, 75])
torch.Size([47, 75])
torch.Size([48, 75])
t